In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

# loading the data

In [2]:

 # load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r' )
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [3]:
 # load document
filename = 'hondo_yehumambo_dset.txt'
doc = load_doc(filename)
print(doc[:200])

"Mwana wangu zvandakudana kudai ndine nyaya yandinoda kutaura newe. Ungazvinetsa hako kuti sei ndakumutsa kuseni kuno asi zvine chirevo." Vakadaro baba vangu vaPfupajena. Ndakaramba ndakanyarara ndich


In [4]:
doc

'"Mwana wangu zvandakudana kudai ndine nyaya yandinoda kutaura newe. Ungazvinetsa hako kuti sei ndakumutsa kuseni kuno asi zvine chirevo." Vakadaro baba vangu vaPfupajena. Ndakaramba ndakanyarara ndichida kunzwa zvaizotevera.\nNdakanga ndamutswa mambakwedza chaiwo apo ndakanga ndaita zvekutumirwa nhume. Kubva ndichiberekwa baba vangu vakanga vasati vambondimutsa nguva dzakadai. Kazhinji ndaimuka nguva dzafamba sezvo ndakanga ndisina chekumhanyira.\n"Mwana wangu zvaunondiona kudai handizivi zvirimberi. Hapana anoziva zuva rake rekusanganiswa nemadzitateguru asi dzimwe nguva unogona kuroteswawo." Ndakaramba ndakanyarara ndichiedza kubatanidza nyaya yaitaurwa nababa wangu.\nVakabva vaita zvekumuka pavakanga vakarara apo vakanga vazoita zvekugara. Ini ndakagarawo pandakanga ndarairwa kunzi ndigare. Ndakabva ndateya nzeve kuti ndinyatonzwa zvaida kutaurwa nababa.\n"Nyaya yandinayo ndaikwanisa kuudza madzikoma ako pamwe nevanin\'ina asi ndasarudza iwe. Kunyangwe amai vako handina kumbovaudza

# data cleaning

In [5]:
import re


# Function to clean text and remove numbers
def clean_text(text):
    # Remove numbers using regular expression
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces and newline characters
    text = ' '.join(text.split())

    return text

# Clean the text
cleaned_text = clean_text(doc)

# Print the cleaned text
print(cleaned_text[:200])

"Mwana wangu zvandakudana kudai ndine nyaya yandinoda kutaura newe. Ungazvinetsa hako kuti sei ndakumutsa kuseni kuno asi zvine chirevo." Vakadaro baba vangu vaPfupajena. Ndakaramba ndakanyarara ndich


# tokenizing and vocabulary_size

In [6]:
from keras.preprocessing.text import Tokenizer

# Tokenize and preprocess
tokenizer = Tokenizer()
tokenizer.fit_on_texts([cleaned_text])
# saving the tokenizer for predict function.
pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))
total_words = len(tokenizer.word_index) + 1


In [7]:
sequence_data = tokenizer.texts_to_sequences([doc])[0]
sequence_data[:15]

[62, 15, 1175, 231, 312, 18, 703, 68, 196, 1176, 313, 1, 48, 1177, 314]

# word_embedding

In [8]:
from gensim.models import Word2Vec

# Train word2vec model with gensim
sentences = [sentence.split() for sentence in cleaned_text.split('.')]
model_gensim = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model_gensim.save("word2vec.model")


# building the first_model_as_model1

In [9]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model1 = Sequential()
model1.add(Embedding(total_words, 100, input_length=5))
model1.add(Bidirectional(LSTM(150, return_sequences=True)))
model1.add(Dropout(0.2))
model1.add(LSTM(100))
model1.add(Dense(total_words, activation='softmax'))

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 100)            366100    
                                                                 
 bidirectional (Bidirection  (None, 5, 300)            301200    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 5, 300)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 3661)              369761    
                                                                 
Total params: 1197461 (4.57 MB)
Trainable params: 1197461 (4.57 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

# the_second_model_as_model2

In [11]:
# Load the word2vec model
import numpy as np
model_gensim = Word2Vec.load("word2vec.model")
embedding_matrix = np.zeros((total_words, 100))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = model_gensim.wv[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except KeyError:
        # Word not present in gensim model, a zero embedding will be used for this word
        pass


model2 = Sequential()
model2.add(Embedding(total_words, 100, weights=[embedding_matrix], input_length=5, trainable=False))
model2.add(Bidirectional(LSTM(150, return_sequences=True)))
model2.add(Dropout(0.2))
model2.add(LSTM(100))
model2.add(Dense(total_words, activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 100)            366100    
                                                                 
 bidirectional_1 (Bidirecti  (None, 5, 300)            301200    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 5, 300)            0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               160400    
                                                                 
 dense_1 (Dense)             (None, 3661)              369761    
                                                                 
Total params: 1197461 (4.57 MB)
Trainable params: 831361 (3.17 MB)
Non-trainable params: 366100 (1.40 MB)
______________

In [13]:
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

input_sequences = []
for line in doc.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences = np.array(pad_sequences(input_sequences, maxlen=6, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = keras.utils.to_categorical(y, num_classes=total_words)





# training_the_models

In [14]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)

In [15]:
# Training Model 1
history1 = model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

Epoch 1/100
164/164 [==============================] - 120s 359ms/step - loss: 7.8776 - accuracy: 0.0334 - val_loss: 7.8325 - val_accuracy: 0.0347
Epoch 2/100
164/164 [==============================] - 46s 277ms/step - loss: 7.2805 - accuracy: 0.0344 - val_loss: 8.0210 - val_accuracy: 0.0347
Epoch 3/100
164/164 [==============================] - 48s 292ms/step - loss: 7.0440 - accuracy: 0.0344 - val_loss: 8.2202 - val_accuracy: 0.0306
Epoch 4/100
164/164 [==============================] - 52s 316ms/step - loss: 6.8507 - accuracy: 0.0353 - val_loss: 8.4490 - val_accuracy: 0.0301
Epoch 5/100
164/164 [==============================] - 54s 330ms/step - loss: 6.6620 - accuracy: 0.0357 - val_loss: 8.7256 - val_accuracy: 0.0275
Epoch 6/100
164/164 [==============================] - 54s 332ms/step - loss: 6.4409 - accuracy: 0.0374 - val_loss: 8.9528 - val_accuracy: 0.0241
Epoch 7/100
164/164 [==============================] - 57s 345ms/step - loss: 6.2101 - accuracy: 0.0409 - val_loss: 9.2463 

164/164 [==============================] - 63s 385ms/step - loss: 0.4703 - accuracy: 0.9074 - val_loss: 13.2317 - val_accuracy: 0.0587
Epoch 57/100
164/164 [==============================] - 63s 383ms/step - loss: 0.4548 - accuracy: 0.9036 - val_loss: 13.3165 - val_accuracy: 0.0576
Epoch 58/100
164/164 [==============================] - 63s 384ms/step - loss: 0.4398 - accuracy: 0.9057 - val_loss: 13.3405 - val_accuracy: 0.0619
Epoch 59/100
164/164 [==============================] - 62s 378ms/step - loss: 0.4386 - accuracy: 0.9057 - val_loss: 13.4135 - val_accuracy: 0.0584
Epoch 60/100
164/164 [==============================] - 62s 379ms/step - loss: 0.4181 - accuracy: 0.9080 - val_loss: 13.4635 - val_accuracy: 0.0610
Epoch 61/100
164/164 [==============================] - 63s 386ms/step - loss: 0.4072 - accuracy: 0.9095 - val_loss: 13.5172 - val_accuracy: 0.0616
Epoch 62/100
164/164 [==============================] - 62s 376ms/step - loss: 0.3943 - accuracy: 0.9110 - val_loss: 13.5421 

In [16]:
# Training Model 2
history2 = model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

Epoch 1/100
164/164 [==============================] - 225s 441ms/step - loss: 7.8773 - accuracy: 0.0332 - val_loss: 7.8318 - val_accuracy: 0.0347
Epoch 2/100
164/164 [==============================] - 46s 281ms/step - loss: 7.3369 - accuracy: 0.0344 - val_loss: 8.0590 - val_accuracy: 0.0347
Epoch 3/100
164/164 [==============================] - 47s 285ms/step - loss: 7.2524 - accuracy: 0.0344 - val_loss: 8.2459 - val_accuracy: 0.0347
Epoch 4/100
164/164 [==============================] - 48s 293ms/step - loss: 7.2264 - accuracy: 0.0344 - val_loss: 8.3456 - val_accuracy: 0.0347
Epoch 5/100
164/164 [==============================] - 48s 294ms/step - loss: 7.2145 - accuracy: 0.0344 - val_loss: 8.4655 - val_accuracy: 0.0347
Epoch 6/100
164/164 [==============================] - 48s 296ms/step - loss: 7.2044 - accuracy: 0.0344 - val_loss: 8.5445 - val_accuracy: 0.0347
Epoch 7/100
164/164 [==============================] - 50s 308ms/step - loss: 7.1991 - accuracy: 0.0344 - val_loss: 8.5946 

164/164 [==============================] - 52s 320ms/step - loss: 4.0486 - accuracy: 0.1350 - val_loss: 11.4419 - val_accuracy: 0.0312
Epoch 57/100
164/164 [==============================] - 55s 335ms/step - loss: 3.9940 - accuracy: 0.1491 - val_loss: 11.4261 - val_accuracy: 0.0284
Epoch 58/100
164/164 [==============================] - 52s 319ms/step - loss: 3.9242 - accuracy: 0.1523 - val_loss: 11.5208 - val_accuracy: 0.0309
Epoch 59/100
164/164 [==============================] - 52s 320ms/step - loss: 3.8918 - accuracy: 0.1552 - val_loss: 11.4652 - val_accuracy: 0.0301
Epoch 60/100
164/164 [==============================] - 53s 324ms/step - loss: 3.8310 - accuracy: 0.1701 - val_loss: 11.5486 - val_accuracy: 0.0292
Epoch 61/100
164/164 [==============================] - 52s 319ms/step - loss: 3.7908 - accuracy: 0.1781 - val_loss: 11.5884 - val_accuracy: 0.0304
Epoch 62/100
164/164 [==============================] - 53s 322ms/step - loss: 3.7341 - accuracy: 0.1856 - val_loss: 11.6180 

In [17]:
val_loss_model1 = history1.history['val_loss'][-1]
val_loss_model2 = history2.history['val_loss'][-1]

print(f"Validation Loss for Model 1: {val_loss_model1}")
print(f"Validation Loss for Model 2: {val_loss_model2}")

Validation Loss for Model 1: 14.827579498291016
Validation Loss for Model 2: 13.31383991241455


In [18]:
if val_loss_model1 < val_loss_model2:
    best_model = model1
    best_model_name = "best_model1.h5"
else:
    best_model = model2
    best_model_name = "best_model2.h5"

best_model.save(best_model_name)
print(f"Saved the best model as {best_model_name}")

C:\Users\PC\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Saved the best model as best_model2.h5


In [19]:
params_model1 = model1.count_params()
params_model2 = model2.count_params()

print(f"Model 1 has {params_model1} parameters.")
print(f"Model 2 has {params_model2} parameters.")

Model 1 has 1197461 parameters.
Model 2 has 1197461 parameters.


In [25]:
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the previously saved model
model = load_model('best_model2.h5')

def predict_next_words(model, tokenizer, text, num_words=1):
    """
    Predict the next set of words using the trained model.

    Args:
    - model (keras.Model): The trained model.
    - tokenizer (Tokenizer): The tokenizer object used for preprocessing.
    - text (str): The input text.
    - num_words (int): The number of words to predict.

    Returns:
    - str: The predicted words.
    """
    for _ in range(num_words):
        # Tokenize and pad the text
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = pad_sequences([sequence], maxlen=5, padding='pre')

        # Predict the next word
        predicted_probs = model.predict(sequence, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)

        # Convert the predicted word index to a word
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break

        # Append the predicted word to the text
        text += " " + output_word

    return ' '.join(text.split(' ')[-num_words:])


# Prompt the user for input
user_input = input("Please type five words in Shona: ")

# Predict the next words
predicted_words = predict_next_words(model, tokenizer, user_input, num_words=3)
print(f"The next words might be: {predicted_words}")

Please type five words in Shona: mangwana ndikamuka ndirikuda kuenda newe
The next words might be: nguva ataurwa kwaisangotenderwa
